In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor


In [2]:
with open('cars.pkl', 'rb') as carpickle:
    cars_df=pickle.load(carpickle)

In [3]:
with open('cars_witthout_drive_size_paint_NaN.pkl', 'rb') as carpickles:
    cars_limited_df=pickle.load(carpickles)

In [4]:
ohe=OneHotEncoder(categories='auto', sparse=False, drop='first')
ss=StandardScaler()

In [5]:
def onehotencoder(column_name, df):
    column_name_ohe = df[[column_name]]
    ohe = OneHotEncoder(categories = 'auto', sparse = False)
    ohe.fit(column_name_ohe)
    column_transformed = ohe.transform(column_name_ohe)
    column_encoded_ohe = pd.DataFrame(column_transformed, columns = ohe.get_feature_names([column_name]), index = df.index)
    return column_encoded_ohe

In [6]:
a = onehotencoder('region' , cars_df)
b = onehotencoder('manufacturer' , cars_df)
# c = onehotencoder('model' , df_3)
d = onehotencoder('condition' , cars_df)
e = onehotencoder('cylinders' , cars_df)
f = onehotencoder('fuel' , cars_df)
g = onehotencoder('title_status' , cars_df)
h = onehotencoder('transmission' , cars_df)
i = onehotencoder('drive' , cars_df)
j = onehotencoder('size' , cars_df)
k = onehotencoder('type', cars_df)
l = onehotencoder('paint_color' , cars_df)
m = onehotencoder('state', cars_df)

In [7]:
# X_train_ohe = X_train.select_dtypes('object')
# X_train_encoded = ohe.fit_transform(X_train_ohe)
# train_df = pd.DataFrame(X_train_encoded, columns=ohe.get_feature_names(), index=X_train.index)

In [7]:
df_final1 = pd.concat([cars_df,a,b,d,e,f,g,h,i,j,k,l,m], axis=1)

In [8]:
df_final1.drop(['drive','region', 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission', 'size', 'type', 'state', 'paint_color'], axis=1, inplace=True)

In [9]:
col_ss = ['year']
ss = StandardScaler()
Scaled = ss.fit_transform(df_final1[col_ss])
Scaled = pd.DataFrame(Scaled, index=df_final1.index)
Scaled.columns = col_ss
df_final1['year'] = Scaled['year']

In [10]:
col_sss = ['odometer']
Scaledd = ss.fit_transform(df_final1[col_sss])
Scaledd = pd.DataFrame(Scaledd, index=df_final1.index)
Scaledd.columns = col_sss
df_final1['odometer'] = Scaledd['odometer']

In [11]:
df_final1['price'] = np.log(df_final1['price'])

In [13]:
df_final1.head()

,price,year,model,odometer,region_SF bay area,region_abilene,region_akron / canton,region_albany,region_albuquerque,region_altoona-johnstown,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
31,9.615805,0.132304,f-150 xlt,0.309363,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,9.898475,-1.877889,f250 super duty,-0.206995,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59,9.546813,-0.091050,odyssey,-0.116632,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,9.615805,1.025724,charger rt 4dr sedan,-0.181177,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,8.006368,-1.877889,town & country,0.930851,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
X= df_final1.drop(['model','price'], axis=1)
y= df_final1['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train, y_train)
print(dummy_regr.score(X_train, y_train))
print(dummy_regr.score(X_test, y_test))

0.0
-2.8136236054576713e-05


In [13]:
SimpleLinearRegression = LinearRegression()
SimpleLinearRegression.fit(X_train, y_train)
print(SimpleLinearRegression.score(X_train,y_train))

0.6499178472438278


In [15]:
print(SimpleLinearRegression.score(X_test, y_test))

-5982575508889466.0


In [17]:
y_hat_train = SimpleLinearRegression.predict(X_train)
y_hat_test = SimpleLinearRegression.predict(X_test)
train_mse_s = mean_squared_error(y_train, y_hat_train)
test_mse_s = mean_squared_error(y_test, y_hat_test)
test_rmse_s = np.sqrt(test_mse_s)
print('Train Mean Squarred Error:', train_mse_s)
print('Test Mean Squarred Error:', test_mse_s)
print('Test RMSE: ', test_rmse_s)

Train Mean Squarred Error: 0.2507149623747856
Test Mean Squarred Error: 4310797466297224.0
Test RMSE:  65656663.533088736


In [19]:
a = onehotencoder('region' , cars_limited_df)
b = onehotencoder('manufacturer' , cars_limited_df)
# c = onehotencoder('model' , df_3)
d = onehotencoder('condition' , cars_limited_df)
e = onehotencoder('cylinders' , cars_limited_df)
f = onehotencoder('fuel' , cars_limited_df)
g = onehotencoder('title_status' , cars_limited_df)
h = onehotencoder('transmission' , cars_limited_df)
# i = onehotencoder('drive' , cars_limited_df)
# j = onehotencoder('size' , cars_limited_df)
k = onehotencoder('type', cars_limited_df)
# l = onehotencoder('paint_color' , cars_limited_df)
m = onehotencoder('state', cars_limited_df)

In [20]:
df_final2 = pd.concat([cars_limited_df,a,b,d,e,f,g,h,k,m], axis=1)

In [21]:
df_final2.drop(['region', 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission','type', 'state'], axis=1, inplace=True)

In [22]:
df_final2.isna().sum()

price                 0
year                  0
model                 0
odometer              0
region_SF bay area    0
                     ..
state_vt              0
state_wa              0
state_wi              0
state_wv              0
state_wy              0
Length: 541, dtype: int64

In [23]:
df_final2['price'] = np.log(df_final2['price'])

scaledddddd = ss.fit_transform(df_final2[['odometer']].values)
df_final2['odometer']= scaledddddd

scaleddddd = ss.fit_transform(df_final2[['year']].values)
df_final2['year']= scaleddddd

In [24]:
X1= df_final2.drop(['model','price'], axis=1)
y1= df_final2['price']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [25]:
SimpleLinearRegression.fit(X_train1, y_train1)
print(SimpleLinearRegression.score(X_train1,y_train1))

0.5262638320804208


In [26]:
print(SimpleLinearRegression.score(X_test1, y_test1))

-45284727988465.38


In [27]:
y_hat_train1 = SimpleLinearRegression.predict(X_train1)
y_hat_test1 = SimpleLinearRegression.predict(X_test1)
train_mse_s1 = mean_squared_error(np.exp(y_train1), y_hat_train1)
test_mse_s1 = mean_squared_error(np.exp(y_test1), y_hat_test1)
test_rmse_s1 = np.sqrt(test_mse_s1)
print('Train Mean Squarred Error:', train_mse_s1)
print('Test Mean Squarred Error:', test_mse_s1)
print('Test RMSE: ', test_rmse_s1)

Train Mean Squarred Error: 495143077.2019933
Test Mean Squarred Error: 40377595184913.5
Test RMSE:  6354336.722657488


In [28]:
cars_limited_df.columns

Index(['region', 'price', 'year', 'manufacturer', 'model', 'condition',
       'cylinders', 'fuel', 'odometer', 'title_status', 'transmission', 'type',
       'state'],
      dtype='object')

In [29]:
from statsmodels.formula.api import ols

formula = 'price ~ C(region) + year + C(manufacturer) + C(condition) + C(cylinders) + C(fuel) + odometer + C(title_status) + C(transmission) + C(type) + C(state)'

model_ols = ols(formula=formula, data=cars_limited_df).fit()
model_summary = model_ols.summary()
model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.708
Model:                            OLS   Adj. R-squared:                  0.707
Method:                 Least Squares   F-statistic:                     1419.
Date:                Thu, 20 Jan 2022   Prob (F-statistic):               0.00
Time:                        13:15:22   Log-Likelihood:            -3.0020e+06
No. Observations:              292719   AIC:                         6.005e+06
Df Residuals:                  292219   BIC:                         6.010e+06
Df Model:                         499                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                               -3.088e+06   6690.835   -461.570      0.000    -3.1e+06   -3.08e+06
C(region)[T.abilene]                    -4975.3807   4075.103     -1.221      0.222    -1.3e+04    3011.708
C(region)[T.akron / canton]             -2754.2153    907.667     -3.034      0.002   -4533.217    -975.213
C(region)[T.albany]                     -3545.8971    856.090     -4.142      0.000   -5223.809   -1867.985
C(region)[T.albuquerque]                -8844.8260    279.383    -31.658      0.000   -9392.408   -8297.244
C(region)[T.altoona-johnstown]          -2869.4939   5732.945     -0.501      0.617   -1.41e+04    8366.919
C(region)[T.amarillo]                   -6343.7818   4053.774     -1.565      0.118   -1.43e+04    1601.502
C(region)[T.ames]                       -6923.0533    707.175     -9.790      0.000   -8309.096   -5537.010
C(region)[T.anchorage / mat-su]         -6.513e+04    245.891   -264.883      0.000   -6.56e+04   -6.47e+04
C(region)[T.ann arbor]                  -6800.3315    393.624    -17.276      0.000   -7571.823   -6028.840
C(region)[T.annapolis]                  -8017.4333   4131.586     -1.941      0.052   -1.61e+04      80.359
C(region)[T.appleton-oshkosh-FDL]       -4297.8263   2126.639     -2.021      0.043   -8465.979    -129.673
C(region)[T.asheville]                  -3488.8742   2896.079     -1.205      0.228   -9165.108    2187.360
C(region)[T.ashtabula]                  -2445.4486   1007.827     -2.426      0.015   -4420.762    -470.136
C(region)[T.athens]                     -3219.1388    929.026     -3.465      0.001   -5040.005   -1398.273
C(region)[T.atlanta]                    -4657.8190    885.534     -5.260      0.000   -6393.442   -2922.196
C(region)[T.auburn]                     -7151.3833    655.367    -10.912      0.000   -8435.884   -5866.882
C(region)[T.augusta]                    -4146.9275    969.078     -4.279      0.000   -6046.292   -2247.563
C(region)[T.austin]                     -4514.7908   4045.363     -1.116      0.264   -1.24e+04    3414.009
C(region)[T.bakersfield]                 2767.3861    249.627     11.086      0.000    2278.124    3256.648
C(region)[T.baltimore]                  -8865.4845   4097.424     -2.164      0.030   -1.69e+04    -834.648
C(region)[T.baton rouge]                -5406.2910    750.777     -7.201      0.000   -6877.793   -3934.789
C(region)[T.battle creek]               -7800.6119    384.618    -20.281      0.000   -8554.452   -7046.772
C(region)[T.beaumont / port arthur]     -4978.2836   4069.919     -1.223      0.221    -1.3e+04    2998.645
C(region)[T.bellingham]                 -7859.0440    377.113    -20.840      0.000   -8598.176   -7119.912
C(region)[T.bemidji]                    -5141.6462    966.9

In [30]:
# ,d,e,f,g,h,k,m
df_f = pd.concat([cars_limited_df,a,b,d,e,f,g,h,k,m],axis=1)
df_f.isna().sum()

region          0
price           0
year            0
manufacturer    0
model           0
               ..
state_vt        0
state_wa        0
state_wi        0
state_wv        0
state_wy        0
Length: 550, dtype: int64

In [31]:
df_f.drop(['region', 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission','type', 'state'], axis=1, inplace=True)
df_f.isna().sum()

price                 0
year                  0
model                 0
odometer              0
region_SF bay area    0
                     ..
state_vt              0
state_wa              0
state_wi              0
state_wv              0
state_wy              0
Length: 541, dtype: int64

In [32]:
scaleddddd = ss.fit_transform(df_f[['year']].values)
df_f['year']= scaleddddd
df_f.isna().sum()

price                 0
year                  0
model                 0
odometer              0
region_SF bay area    0
                     ..
state_vt              0
state_wa              0
state_wi              0
state_wv              0
state_wy              0
Length: 541, dtype: int64

In [33]:
scaledddddd = ss.fit_transform(df_final2[['odometer']].values)
df_final2['odometer']= scaledddddd
df_f.isna().sum()

price                 0
year                  0
model                 0
odometer              0
region_SF bay area    0
                     ..
state_vt              0
state_wa              0
state_wi              0
state_wv              0
state_wy              0
Length: 541, dtype: int64